참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengege Learning, 2013.

`python` 에서 수학 관련 기능을 좀 더 쉽게 사용할 수 있도록 해 주는 `module`을 불러 들임 (모든 기능이 다 사용되지는 않을 수도 있음)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능

## 예제 02.007

### 문제에서 주어진 매개변수

강 봉의 단면적

In [ ]:
A_bar_mm2 = 900

강 봉의 갯수

In [ ]:
n_bar = 4

압축 하중

In [ ]:
P_N = 1000e3

강과 콘크리트의 탄성계수

In [ ]:
E_steel_Pa = 200e9
E_concrete_Pa = 14e9

콘크리트 정사각형 기둥의 한 변의 길이

In [ ]:
w_mm = 300

### 풀이

강 봉의 변형과 콘크리트 기둥의 변형은 같다고 가정한다.
$$\delta_{steel}=\delta_{concrete}$$

강 봉에 걸리는 하중과 콘크리트에 걸리는 하중의 합이 압축하중이 된다.
$$P=P_{steel} + P_{concrete}$$

강 봉의 길이와 콘크리트 기둥의 길이가 같다면, 변형률도 같다.
$$\epsilon_{steel}=\epsilon_{concrete}$$

탄성계수를 적용하면 강 봉과 콘크리트의 압축 응력을 구할 수 있다.
$$\epsilon_{steel}=\frac{\sigma_{steel}}{E_{steel}}=\frac{\sigma_{concrete}}{E_{concrete}}=\epsilon_{concrete}$$

한편, 하중을 응력으로 표시할 수 있다.
$$P=\sigma_{steel}A_{steel}+\sigma_{concrete}A_{concrete}$$

각 면적은 다음과 같이 구할 수 있다.
$$A_{steel} = n_{bar}A_{bar}$$
$$A_{concrete} = w^2 - A_{steel}$$

하중 식은 다음과 같이 될 것이다.
$$P=\sigma_{steel}n_{bar}A_{bar}+\sigma_{concrete}(w^2 - n_{bar}A_{bar})$$

하중 식과 응력 식을 모으면 다음과 같다.
\begin{equation}
\begin{array}{}
    n_{bar}A_{bar}\sigma_{steel}+(w^2 - n_{bar}A_{bar})\sigma_{concrete}&=&P\\
    \frac{1}{E_{steel}}\sigma_{steel}-\frac{1}{E_{concrete}}\sigma_{concrete}&=&0\\
\end{array}
\end{equation}

행렬 형태로도 정리할 수 있다.
\begin{equation}
\begin{bmatrix}
n_{bar}A_{bar} & w^2 - n_{bar}A_{bar} \\
\frac{1}{E_{steel}} & -\frac{1}{E_{concrete}}
\end{bmatrix}
\begin{bmatrix}
\sigma_{steel} \\
\sigma_{concrete}
\end{bmatrix}
=
\begin{bmatrix}
P\\
0
\end{bmatrix}
\end{equation}

이런 형태의 문제는 다음과 같이 줄여 쓸 수도 있다.
$$Ax=b \\ x = A^{-1}b$$

`python` 변수 $A_{bar}$ 와 $w$ 의 단위를 변경한다.

In [ ]:
X = 1e-3
A_bar_m2 = A_bar_mm2 * X**2
w_m = w_mm * X

계수 행렬 $A$를 정한다. 

In [ ]:
A_mat = np.matrix([[n_bar * A_bar_m2, w_m**2 - n_bar * A_bar_m2],
                   [1./E_steel_Pa, -1./E_concrete_Pa]])

상수 벡터 $b$를 정한다.

In [ ]:
b_vec = np.matrix([[P_N],
                  [0.0]])

위 행렬 방정식을 푼다.

In [ ]:
sigma_Pa_mat = na.solve(A_mat, b_vec)
sigma_Pa_mat

MPa 단위로는 다음과 같다.

In [ ]:
sigma_MPa_mat = sigma_Pa_mat * 1e-6
sigma_MPa_mat

### 확인

강 봉 1개가 담당하는 하중은 다음과 같을 것이다.

In [ ]:
P_steel_1_kN = sigma_Pa_mat[0] * A_bar_m2 * 1e-3
P_steel_1_kN

강 봉 4개가 담당하는 하중은 다음과 같을 것이다.

In [ ]:
P_steel_4_kN = P_steel_1_kN * n_bar
P_steel_4_kN

콘크리트가 담당하는 하중은 다음과 같을 것이다.

In [ ]:
P_concrete_kN = sigma_Pa_mat[1] * (w_m**2 - n_bar * A_bar_m2) * 1e-3
P_concrete_kN

확인을 위해 위 두 하중을 더해 본다.

In [ ]:
float(P_steel_4_kN + P_concrete_kN)

아래는 프로그램이 맞게 작동했는지 확인하기 위해 추가된 부분이다. 

참고 : Francesco Mosconi, Travis + Anaconda + Jupyter, 2017 Aug 09, [Online] Available: https://github.com/ghego/travis_anaconda_jupyter.

In [ ]:
zero_mat = A_mat * sigma_Pa_mat - b_vec
assert(1e-7 > abs(zero_mat[0, 0]))
assert(1e-7 > abs(zero_mat[1, 0]))

In [ ]:
assert(1e-7 > (abs(float(P_steel_4_kN + P_concrete_kN) - P_N * 1e-3)))